In [1]:
import sys

sys.path.append("..")

from logger import get_logger

%load_ext autoreload
%autoreload 2

In [2]:
logger = get_logger(__name__)
logger.debug("This is a debug message")

DEBUG:__main__:This is a debug message


In [3]:
import torch

In [8]:
x = torch.nn.Parameter(torch.randn(5, 5), requires_grad=True)

y = 5 * x + 2

vals, indices = torch.topk(y, 2)

print(vals.requires_grad, indices.requires_grad)

loss = vals.norm()
loss.backward()

True False


In [6]:
vals._grad

/tmp/ipykernel_462449/3846127606.py:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at aten/src/ATen/core/TensorBody.h:489.)
  vals._grad
